In [5]:
import pandas as pd
import numpy as np
import datetime

import sys
import os

current = os.path.dirname(os.path.realpath("Test.ipynb"))
parent = os.path.dirname(current)
sys.path.append(parent + "\\Functions\\")
import UsefulFunctions as UF
import ImageFunctions as IF
root = UF.DataRoot(2)

# read in fts from csv
df_all = pd.read_csv(root + "Aaron\ProstateMRL\Data\Paper1\Features\All_fts_pVol.csv")

print(df_all["PatID"].unique())

[ 653  713  752  826 1088 1089 1118 1303 1307 1464 1029 1302 1431 1481
 1540 1553 1601 1642  829  955]


In [2]:
import pandas as pd
import numpy as np
import datetime

import sys
import os

current = os.path.dirname(os.path.realpath("Test.ipynb"))
parent = os.path.dirname(current)
sys.path.append(parent + "\\Functions\\")
import UsefulFunctions as UF
import ImageFunctions as IF

In [ ]:
# read in patient key
root = UF.DataRoot(2)
LimbusKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey.csv")
AllKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\AllPatientKey_s.csv")

# Loop through treatments
treatments = AllKey["Treatment"].unique()
new_key = pd.DataFrame()
for t in treatments:
    AllKey_t = AllKey[AllKey["Treatment"] == t]
    LimbusKey_t = LimbusKey[LimbusKey["Treatment"] == t]

    # Loop through patients
    patIDs = LimbusKey_t["PatID"].unique()

    for pat in patIDs:
        pat_L = LimbusKey_t[LimbusKey_t["PatID"] == pat]
        pat_A = AllKey_t[AllKey_t["PatID"] == pat]
        print(pat_A)
        print(pat_L)
        # add date and days column from all to limbus key based on Scan
        for i in range(len(pat_L)):
            scan = pat_L.iloc[i]["Scan"]
            fraction = pat_A.iloc[i]["Fraction"]
            date = pat_A[pat_A["Scan"] == scan]["Date"].values[0]
            days = pat_A[pat_A["Scan"] == scan]["Days"].values[0]
            pat_L.at[pat_L.index[i], "Date"] = str(date)
            pat_L.at[pat_L.index[i], "Days"] = int(days)
            pat_L.at[pat_L.index[i], "Fraction"] = int(fraction)

        pat_L["Days"] = pat_L["Days"].astype(int)
        # sort by days
        pat_L = pat_L.sort_values(by=["Fraction"])

    # add to new key
        new_key = new_key.append(pat_L)

new_key.to_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey_s.csv", index=False)


        
        



In [20]:
import pandas as pd
import numpy as np
import os

import sys

current = os.path.dirname(os.path.realpath(''))
parent = os.path.dirname(current)
sys.path.append(parent + "\\Functions\\")
import UsefulFunctions as UF
#import ImageFunctions as IF

root = UF.DataRoot(2)

# read in patient key
PatKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\AllPatientKey_s.csv")
PatKey = PatKey[PatKey["Treatment"] == "SABR"]

# read in ft vals
df_all = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Data\\Paper1\\Limbus\\SABR_fts_all.csv")


df_all = df_all.drop(columns = [col for col in df_all.columns if "diagnostics" in col])
df_all = df_all.drop(columns = [col for col in df_all.columns if "Unnamed" in col])

fts = df_all.columns[2:]

fractions = PatKey["Fraction"].unique()
patIDs = PatKey["PatID"].unique()

# merge with patient key to get date and days and fraction
df_all_m = pd.merge(df_all, PatKey[["PatID", "Scan", "Days", "Fraction"]],on = ["PatID", "Scan"], how = "left")

frac = df_all_m.pop("Fraction")
df_all_m.insert(2, "Fraction", frac)

days = df_all_m.pop("Days")
df_all_m.insert(3, "Days", days)


df_all_m["Days"] = df_all_m["Days"].astype(int)
df_all_m["Fraction"] = df_all_m["Fraction"].astype(int)

df_all_m = df_all_m.sort_values(by = ["PatID", "Fraction", "Days"])
df_all_m.to_csv(root + "\\Aaron\\ProstateMRL\\Data\\Paper1\\Limbus\\SABR_fts_all_m.csv", index = False)




In [23]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

csvs = os.listdir("E:\\Aaron\ProstateMRL\Data\Paper1\Longitudinal\Clustering\\Labels\\")
csvs = [csv for csv in csvs if "HM" in csv]

fts_s = pd.read_csv("E:\\Aaron\ProstateMRL\Data\Paper1\Longitudinal\Clustering\\HM_SelectedFeatures.csv")
fts_s = fts_s["Feature"].values
for csv in csvs:
    df = pd.read_csv("E:\\Aaron\ProstateMRL\Data\Paper1\Longitudinal\Clustering\\Labels\\" + csv)
    pat = csv.split("_")[1][:-4]
    df["Selected"] = df["Feature"].apply(lambda x: x in fts_s)
    # print where selected is True
    df = df[['Feature', 'Cluster', 'Fraction', 'FeatureChange', 'Selected']]
    df['Feature'] = df['Feature'].str.replace('original_', '')
    clusters = df["Cluster"].unique()
    for c in clusters:
        df_c = df[df["Cluster"] == c]
        df_c = df_c.sort_values(by = ["Fraction"])

        # get selected features
        text_str = df_c[df_c["Selected"] == True]["Feature"].unique()
        text_str = '\n'.join(text_str)
        props = dict(boxstyle='round', facecolor='lightblue', alpha=0.5)
        fts = df_c["Feature"].values
        plt.figure(figsize=(10, 10))
        plt.title(" Cluster " + str(c))
        sns.set_theme(style="whitegrid")
        sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
        for ft in fts:
            df_ft = df_c[df_c["Feature"] == ft]
            values = df_ft["FeatureChange"].values
            fractions = df_ft["Fraction"].values
            colour = "blue" if df_ft["Selected"].values[0] else "grey"
            l = df_ft["Feature"].values[0]
            plt.plot(fractions, values, label = l, color = colour)
            #plt.scatter(fractions, values, color = colour)
        plt.xlabel("Fraction", fontsize = 16)
        plt.ylabel("Feature Change", fontsize = 16)
        plt.xticks(np.arange(1, 5.1, 1))
        plt.xlim(1, 5)
        #plt.ylim(-1, 1)
        # add text box
        plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes, fontsize=14, verticalalignment='top', bbox=props)
        
        #plt.legend(title = "Feature Selected", bbox_to_anchor=(1, 0.6), labels = ["Yes", "No"])
        plt.title("Cluster " + str(c), fontsize = 20)
        plt.savefig("E:\\Aaron\ProstateMRL\Data\Paper1\Longitudinal\Clustering\\Plots\\" + pat + "_Cluster" + str(c) + ".png", bbox_inches = "tight")
        plt.close()






In [ ]:
import os
import matplotlib.pyplot as plt